In [1]:
# importing module
from pymongo import MongoClient
client = MongoClient("mongodb://localhost:27017/")


In [2]:
mydatabase = client['mongodb']

In [5]:
QUERY_A = mydatabase.film.aggregate([
   {
     '$group': {
        '_id': 0,
        'Gesamtanzahl der verfügbaren Filme': { '$sum': 1 }
     }
   },
   {
      '$project': {
         '_id': 0
      }
   }
])


In [6]:
QUERY_A.next()

{'Gesamtanzahl der verfügbaren Filme': 1000}

In [352]:
QUERY_B = mydatabase.inventory.aggregate([
    {"$group": {
        '_id': "$store_id",
        'film_set': {
            '$addToSet': "$film_id"
            }
        }
    },
    {"$project": {
        '_id': 0,
        'Store_ID': "$_id",
        'Verschiedene Filme': {
            '$size': "$film_set"
            }
        }
    }])


In [9]:
QUERY_B.next()

{'Store_ID': 1, 'Verschiedene Filme': 759}

Select actor.actor_id, actor.first_name, Count(*) Anzahl, actor.last_name from film_actor
join actor on film_actor.actor_id = actor.actor_id
group by actor.actor_id
Order by Anzahl DESC 
Limit 10

In [393]:
QUERY_C = mydatabase.actor.aggregate([
   {
    '$lookup':{
        'from': "film_actor",
        'localField': "actor_id",
        'foreignField': "actor_id",
        'as': "actor_info"
    }
   },
   {
    '$project':{
        '_id':0,
        'Vorname':'$first_name',
        'Nachname':'$last_name',
        'Anzahl Auftritte': {
            '$size': "$actor_info"
        }
     }
    },
    {
    '$sort':{
        'Anzahl Auftritte':-1
    }
   },
   {
    '$limit':10
   }
])

In [392]:
QUERY_C.next()

{'Vorname': 'Gina', 'Nachname': 'Degeneres', 'Anzahl Auftritte': 42}

Select sum(payment.amount) total, staff.staff_id, staff.first_name, staff.last_name  from payment
join staff on payment.staff_id = Staff.staff_id
group by staff.staff_id

total	staff_id	first_name	last_name
30252.12	1	Mike	Hillyer
31059.92	2	Jon	Stephens

In [394]:
QUERY_D = mydatabase.payment.aggregate([
    {
        '$group':{
            '_id': "$staff_id",
            'total':{
                '$sum':"$amount"
            }
        }
    },
    {
        '$project':{
            '_id':0,
            'Mitarber ID':"$_id",
            'Erlös': {'$trunc':['$total', 2]}
        }
    }
])

In [396]:
QUERY_D.next()

{'Mitarber ID': 2, 'Erlös': 31059.92}

Select customer_id, Count(*) from rental
group by customer_id
Order By Count(*) DESC
Limit 10;

In [411]:
QUERY_E = mydatabase.rental.aggregate([
    {
        '$group':{
            '_id': "$customer_id",
            'count':{
                '$sum':1
            }
        }
    },
    {
        '$sort':{
             'count' : -1
            } 
    },
    {
        '$limit':10
    },
    {
        '$project':{
            '_id':0,
            'Kunden ID':'$_id',
            'Anzahl Entleihungen':'$count'
        }
    }
])

In [412]:
QUERY_E.next()


{'Kunden ID': 148, 'Anzahl Entleihungen': 46}

Select customer.customer_id, customer.first_name, customer.last_name,
Sum(payment.amount) as total, address.district from payment
join customer on payment.customer_id = customer.customer_id
join store on store.store_id = customer.store_id
join address on store.address_id = address.address_id
Group by customer.customer_id, address.address_id
Order by total DESC
Limit 10

customer_id	first_name	last_name	total	district
148	Eleanor	Hunt	211.55	Alberta
526	Karl	Seal	208.58	QLD
178	Marion	Snyder	194.61	QLD
137	Rhonda	Kennedy	191.62	QLD
144	Clara	Shaw	189.60	Alberta
459	Tommy	Collazo	183.63	Alberta
181	Ana	Bradley	167.67	QLD
410	Curtis	Irby	167.62	QLD
236	Marcia	Dean	166.61	Alberta
403	Mike	Way	162.67	Alberta

In [418]:
QUERY_F = mydatabase.payment.aggregate([
   {
    '$group':{
        '_id': "$customer_id",
        'sum': {
            '$sum':'$amount'
        }
    }
   },
   {
    '$sort':{
        'sum':-1
    }
   },
   {
    '$limit':10
   },
   {
    '$lookup':{
        'from': "customer",
        'localField': "_id",
        'foreignField': "customer_id",
        'as': 'customer_info'
    }
   },
    {
    '$lookup':{
        'from': "store",
        'localField': "customer_info.store_id",
        'foreignField': "store_id",
        'as': 'store_info'
    }
   },
    {
    '$lookup':{
        'from': "address",
        'localField': "store_info.address_id",
        'foreignField': "address_id",
        'as': 'address_info'
     }
    },
    {
    '$unwind': '$customer_info'
    },
    {
    '$unwind': '$address_info'
    },
    {
    '$project':{
        '_id':0,
        'Vorname':'$customer_info.first_name',
        'Nachname':'$customer_info.last_name',
        'Ausgabe': '$sum',
        'Niederlassung': "$address_info.district"
     }
    }
])

In [422]:
QUERY_F.next()

{'Vorname': 'Rhonda',
 'Nachname': 'Kennedy',
 'Ausgabe': 191.62,
 'Niederlassung': 'QLD'}

g)
Select Count(film.film_id) as Amount, film.title from film
join inventory on film.film_id = inventory.film_id
join rental on inventory.inventory_id = rental.inventory_id
Group by film.film_id
Order by Amount DESC

34	Bucket Brotherhood
33	Rocketeer Mother
32	Forward Temple
32	Juggler Hardly
32	Ridgemont Submarine
32	Grit Clockwork
32	Scalawag Duck
31	Robbers Joon
31	Network Peak
31	Timberland Sky

In [494]:
QUERY_G = mydatabase.rental.aggregate([
    {
    '$lookup':{
        'from': "inventory",
        'localField': "inventory_id",
        'foreignField': "inventory_id",
        'as': 'inventory_info'
     }
    },
   {
    '$lookup':{
        'from': "film",
        'localField': "inventory_info.film_id",
        'foreignField': "film_id",
        'as': "film_info"
    }
   },
   {
     '$group':{
        '_id': "$film_info.film_id",
        'title': {
            '$first': "$film_info.title"
        },
        'sum': {
            '$sum': 1
        }
     }
    },
    {
    '$sort':{
        'sum':-1
    }
   },
   {
    '$limit':10
   },
   {
    '$unwind': '$title'
   },
   {
    '$project':{
        '_id':0,
        'Anzahl der Ausleihe': '$sum',
        'Filmname':'$title'
     }
   }
])

In [495]:
QUERY_G.next()

{'Anzahl der Ausleihe': 34, 'Filmname': 'Bucket Brotherhood'}

Select category.name, Count(*) from rental 
join inventory on rental.inventory_id = inventory.inventory_id
join film_category on inventory.film_id = film_category.film_id
join category on film_category.category_id = category.category_id
GROUP BY category.category_id
ORDER BY count DESC
LIMIT 3


name	count
Sports	1179
Animation	1166
Action	1112



In [496]:
QUERY_H = mydatabase.category.aggregate([
    {
    '$lookup':{
        'from': "film_category",
        'localField': "category_id",
        'foreignField': "category_id",
        'as': 'film_category_info'
     }
    },
    {
    '$lookup':{
        'from': "inventory",
        'localField': "film_category_info.film_id",
        'foreignField': "film_id",
        'as': "inventory_info"
    }
   },
   {
    '$lookup':{
        'from': "rental",
        'localField': "inventory_info.inventory_id",
        'foreignField': "inventory_id",
        'as': "rental_info"
    }
   }, 
   {
    '$project':{
        '_id':0,
        'Kategoriename': '$name',
        'Anzahl':{
            '$size':"$rental_info"
        }
    }
   },
   {
    '$sort':{
        'Anzahl': -1
    }
   },
   {
    '$limit':3
   }
])

In [498]:
QUERY_H.next()

{'Kategoriename': 'Animation', 'Anzahl': 1166}

In [ ]:
QUERY_I